<a href="https://colab.research.google.com/github/revirevy/ai-applications/blob/master/Notebooks/Privacy_first_AI_search_using_LangChain_and_Elasticsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Private AI Search with LangChain and Elasticsearch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/blog-langchain-elasticsearch/blob/main/Notebooks/Privacy_first_AI_search_using_LangChain_and_Elasticsearch.ipynb)

Motivations:
* **freshness** - there aren't enough GPUs in the world to train large language models for every problem, data gets old very fast. Semantic search can be used to push context into LLM prompts with real time data.
* **privacy** - pushing our most private of data to the big LLMs isn't really an option when that data is private or the competitive advantage of a company, big or small. Let's use a local smaller LLM that can be deployed privately in a closed network if necessary.

First let's set up the environment


In [1]:
!pip install -q beautifulsoup4 eland elasticsearch huggingface-hub langchain==0.0.157 tqdm requests sentence_transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.6/163.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.6/727.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


In [30]:
%pip install --upgrade --quiet langchain-elasticsearch langchain-openai tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00


# Elastic Cloud and Connection Details

While you can definitely pull this colab down to run lcoally as a python notebook, the simplest way to get this working is to create an Elastic cluster over at  https://cloud.elastic.co/ . Make sure to configure an ML node. You won't need more than the default spec for this project.  Once that cluster is up and running grab your connecting info and edit the below before running it.

In [14]:
# Now we'll load these into the python environment
from getpass import getpass
endpoint = getpass("Elasticsearch Server URL (format: subdomain.etc.gcp.cloud.es.io): ")
# endpoint = getpass("aa0a43c3781045ea86054fb5267e6401.us-central1.gcp.cloud.es.io")
username = getpass("Elasticsearch user: ")
password = getpass("Elasticsearch password: ")
# https://aa0a43c3781045ea86054fb5267e6401.us-central1.gcp.cloud.es.io:443
es_url =  f"https://{username}:{password}@{endpoint}:443"

Elasticsearch Server URL (format: subdomain.etc.gcp.cloud.es.io): ··········
Elasticsearch user: ··········
Elasticsearch password: ··········


In [10]:
import requests
r= requests.get("https://aa0a43c3781045ea86054fb5267e6401.us-central1.gcp.cloud.es.io:443")
r

<Response [401]>

In [11]:
r.json()

{'error': {'root_cause': [{'type': 'security_exception',
    'reason': 'unable to authenticate user [revirevy] for REST request [/]',
    'header': {'WWW-Authenticate': ['Basic realm="security" charset="UTF-8"',
      'Bearer realm="security"',
      'ApiKey']}}],
  'type': 'security_exception',
  'reason': 'unable to authenticate user [revirevy] for REST request [/]',
  'header': {'WWW-Authenticate': ['Basic realm="security" charset="UTF-8"',
    'Bearer realm="security"',
    'ApiKey']}},
 'status': 401}

In [22]:
ES_URL="https://aa0a43c3781045ea86054fb5267e6401.us-central1.gcp.cloud.es.io:443"
ES_URL

'https://aa0a43c3781045ea86054fb5267e6401.us-central1.gcp.cloud.es.io:443'

In [24]:
!curl https://aa0a43c3781045ea86054fb5267e6401.us-central1.gcp.cloud.es.io:443 -H "Authorization: ApiKey SV82Q0k0NEJVaUQ5QmRmU0haWVo6TV9PYzBsZkRRdnlXWVpsWjJ4NW8tdw"

{
  "name" : "instance-0000000000",
  "cluster_name" : "aa0a43c3781045ea86054fb5267e6401",
  "cluster_uuid" : "QwtqPIH-ReeRHtVG9BDy-w",
  "version" : {
    "number" : "8.12.2",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "48a287ab9497e852de30327444b0809e55d46466",
    "build_date" : "2024-02-19T10:04:32.774273190Z",
    "build_snapshot" : false,
    "lucene_version" : "9.9.2",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [16]:
API_KEY="SV82Q0k0NEJVaUQ5QmRmU0haWVo6TV9PYzBsZkRRdnlXWVpsWjJ4NW8tdw=="

In [26]:
!curl "$ES_URL/_cat/indices" \
  -H "Authorization: ApiKey SV82Q0k0NEJVaUQ5QmRmU0haWVo6TV9PYzBsZkRRdnlXWVpsWjJ4NW8tdw" \
  -H "Content-Type: application/json"

green open .elastic-connectors-v1                                            xDpUdcsaQSi2u2IiGEw6cw 1 1  0 0   7.6kb   247b   247b
green open .ds-metrics-fleet_server.agent_versions-default-2024.03.09-000001 wLdeWKGLQRGHF6NSOAUuZw 1 1 39 0 138.8kb 69.4kb 69.4kb
green open .internal.alerts-observability.threshold.alerts-default-000001    MQ3iBhWPSe-ftjRY4FrI9A 1 1  0 0    498b   249b   249b
green open .ds-logs-enterprise_search.api-default-2024.03.09-000001          5urMjsVTSDS2sW7iO7Krng 1 1  3 0  98.4kb 49.2kb 49.2kb
green open .internal.alerts-ml.anomaly-detection.alerts-default-000001       MMjJWyfGRU27y9vqqRfm2Q 1 1  0 0    498b   249b   249b
green open .internal.alerts-security.alerts-default-000001                   AIW6AZEMTP-BO563aQ6OLQ 1 1  0 0    498b   249b   249b
green open .internal.alerts-observability.apm.alerts-default-000001          xWUv7pMARk6DrSZfB0QI-A 1 1  0 0    498b   249b   249b
green open .internal.alerts-observability.metrics.alerts-default-000001      YBaojy

In [ ]:
curl -X POST "${ES_URL}/_bulk?pretty" \
  -H "Authorization: ApiKey "${API_KEY}"" \
  -H "Content-Type: application/json" \
  -d'
{ "index" : { "_index" : "index_name" } }
{"name": "Snow Crash", "author": "Neal Stephenson", "release_date": "1992-06-01", "page_count": 470}
{ "index" : { "_index" : "index_name" } }
{"name": "Revelation Space", "author": "Alastair Reynolds", "release_date": "2000-03-15", "page_count": 585}
{ "index" : { "_index" : "index_name" } }
{"name": "1984", "author": "George Orwell", "release_date": "1985-06-01", "page_count": 328}
{ "index" : { "_index" : "index_name" } }
{"name": "Fahrenheit 451", "author": "Ray Bradbury", "release_date": "1953-10-15", "page_count": 227}
{ "index" : { "_index" : "index_name" } }
{"name": "Brave New World", "author": "Aldous Huxley", "release_date": "1932-06-01", "page_count": 268}
{ "index" : { "_index" : "index_name" } }
{"name": "The Handmaid'"'"'s Tale", "author": "Margaret Atwood", "release_date": "1985-06-01", "page_count": 311}
'

# Scraping a small set of data from Wookieepedia

We'll keep it to two pages for characters active in recent TV shows, too recent for updates to be caught by common 2021 AI data sets.

Check out the original article and origin of this parsing exmaple over at: https://towardsdatascience.com/star-wars-data-science-d32acde3432d

In [3]:
import re
import requests
from bs4 import BeautifulSoup
import pickle
import json
from tqdm import tqdm

In [4]:
scraped = {}
pages = [
    "https://starwars.fandom.com/wiki/N-1_starfighter",
    "https://starwars.fandom.com/wiki/Ahsoka_Tano",
    "https://starwars.fandom.com/wiki/Din_Djarin"]

last_number = 0
for page_url in pages:
    try:

        # Get page
        result = requests.get(page_url)
        content = result.content
        soup = BeautifulSoup(content, "html.parser")

        # Get title
        heading = soup.find('h1', id='firstHeading')
        if heading is None: continue
        heading = heading.text

        # Extract Sidebar
        is_character = False
        side_bar = {}
        sec = soup.find_all('section', class_='pi-item')
        for s in sec:
            title = s.find('h2')
            if title is None:
                title = '<no category>'
            else:
                title = title.text
            side_bar[title] = {}
            items = s.find_all('div', class_='pi-item')
            for item in items:
                attr = item.find('h3', class_='pi-data-label')
                if attr is None:
                    attr = '<no attribute>'
                else:
                    attr = attr.text
                if attr == 'Species': is_character = True
                value = re.sub("[\(\[].*?[\)\]]" ,'', '], '.join(item.find('div', class_='pi-data-value').text.split(']')))
                value = value.strip()[:-1].replace(',,', ',')
                if ',' in value:
                    value = [i.strip() for i in value.split(',') if i.strip() != '']
                side_bar[title][attr] = value

        # Raw page content
        raw_content = soup.find('div', class_='mw-parser-output')
        if raw_content is not None:
            content_pgs = []
            for raw_paragraph in raw_content.find_all('p', recursive=False):
                if 'aside' in str(raw_paragraph): continue
                content_pgs.append(re.sub("[\(\[].*?[\)\]]" ,'', raw_paragraph.text) )
            # paragraph = value = re.sub("[\(\[].*?[\)\]]" ,'', raw_paragraph.text)


        else:
            # Empty page
            paragraph = ''

        # Data object
        scraped[page_url] = {
            'url': page_url,
            'title': heading,
            'is_character': is_character,
            'side_bar': side_bar,
            'paragraph': content_pgs
        }

    except:
        print(f'Failed! {page_url}')


# Save final part to disk
fn =  './starwars_small_canon_data.pickle'
with open(fn, 'wb') as f:
    pickle.dump(scraped, f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
## Let's do a quick test to make sure it worked we. Even if the data is big
## we can chunk it up with the above code and load it in sections.

from pathlib import Path

bookFilePath = "starwars_*_canon_data*.pickle"
files = sorted(Path('.').glob(bookFilePath))
for fn in files:
  with open(fn,'rb') as f:
      part = pickle.load(f)
      for key, value in part.items():
          title = value['title'].strip()
          print(title)

N-1 starfighter
Ahsoka Tano
Din Djarin


# Using LangChain to generate vectors and store in Elasticsearch

First we'll create the embeddings model

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

def setup_embeddings():
    # Huggingface embedding setup
    print(">> Prep. Huggingface embedding setup")
    model_name = "sentence-transformers/all-mpnet-base-v2"
    return HuggingFaceEmbeddings(model_name=model_name)

hf = setup_embeddings()


>> Prep. Huggingface embedding setup


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Next we'll create our elasticsearch vectorstore in the langchain style:

In [39]:
# from langchain.vectorstores import ElasticVectorSearch

# index_name = "book_wookieepedia_small"

# db = ElasticVectorSearch(
#  embedding=hf, elasticsearch_url=ES_URL, index_name=index_name,
#  es_api_key=API_KEY)



from langchain_elasticsearch import ElasticsearchStore #  .vectorstores import Elastcse

index_name = "book_wookieepedia_small"

db = ElasticsearchStore(
  es_url=ES_URL,
  index_name= index_name , # "test_index",
  embedding=hf, es_api_key=API_KEY,
  strategy=ElasticsearchStore.ExactRetrievalStrategy()
)

In [47]:
db.client.indices.__dir__()

['_client',
 '_transport',
 '_client_meta',
 '_headers',
 '_request_timeout',
 '_ignore_status',
 '_max_retries',
 '_retry_on_timeout',
 '_retry_on_status',
 '_verified_elasticsearch',
 '__module__',
 'add_block',
 'analyze',
 'clear_cache',
 'clone',
 'close',
 'create',
 'create_data_stream',
 'data_streams_stats',
 'delete',
 'delete_alias',
 'delete_data_lifecycle',
 'delete_data_stream',
 'delete_index_template',
 'delete_template',
 'disk_usage',
 'downsample',
 'exists',
 'exists_alias',
 'exists_index_template',
 'exists_template',
 'explain_data_lifecycle',
 'field_usage_stats',
 'flush',
 'forcemerge',
 'get',
 'get_alias',
 'get_data_lifecycle',
 'get_data_stream',
 'get_field_mapping',
 'get_index_template',
 'get_mapping',
 'get_settings',
 'get_template',
 'migrate_to_data_stream',
 'modify_data_stream',
 'open',
 'promote_data_stream',
 'put_alias',
 'put_data_lifecycle',
 'put_index_template',
 'put_mapping',
 'put_settings',
 'put_template',
 'recovery',
 'refresh',
 '

In [83]:
from IPython.display import HTML,JSON
import json
from pprint import pprint
jval=db.client.indices.stats()
print(jval)
#.__dict__,)
print(jval)

{'_shards': {'total': 34, 'successful': 34, 'failed': 0}, '_all': {'primaries': {'docs': {'count': 179, 'deleted': 0}, 'shard_stats': {'total_count': 17}, 'store': {'size_in_bytes': 607244, 'total_data_set_size_in_bytes': 607244, 'reserved_in_bytes': 0}, 'indexing': {'index_total': 183, 'index_time_in_millis': 205, 'index_current': 0, 'index_failed': 0, 'delete_total': 4, 'delete_time_in_millis': 7, 'delete_current': 0, 'noop_update_total': 0, 'is_throttled': False, 'throttle_time_in_millis': 0, 'write_load': 2.3046649255828098e-06}, 'get': {'total': 0, 'time_in_millis': 0, 'exists_total': 0, 'exists_time_in_millis': 0, 'missing_total': 0, 'missing_time_in_millis': 0, 'current': 0}, 'search': {'open_contexts': 0, 'query_total': 1795, 'query_time_in_millis': 373, 'query_current': 0, 'fetch_total': 1787, 'fetch_time_in_millis': 27, 'fetch_current': 0, 'scroll_total': 0, 'scroll_time_in_millis': 0, 'scroll_current': 0, 'suggest_total': 0, 'suggest_time_in_millis': 0, 'suggest_current': 0}

In [87]:
JSON(jval.body).__dir__()

['metadata',
 'url',
 'filename',
 '_data',
 '__module__',
 '__doc__',
 '__init__',
 '_check_data',
 'data',
 '_data_and_metadata',
 '_repr_json_',
 '_read_flags',
 '_show_mem_addr',
 '__repr__',
 'reload',
 '__dict__',
 '__weakref__',
 '__new__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [90]:
JSON(jval.body)._repr_json_()

({'_shards': {'total': 34, 'successful': 34, 'failed': 0},
  '_all': {'primaries': {'docs': {'count': 179, 'deleted': 0},
    'shard_stats': {'total_count': 17},
    'store': {'size_in_bytes': 607244,
     'total_data_set_size_in_bytes': 607244,
     'reserved_in_bytes': 0},
    'indexing': {'index_total': 183,
     'index_time_in_millis': 205,
     'index_current': 0,
     'index_failed': 0,
     'delete_total': 4,
     'delete_time_in_millis': 7,
     'delete_current': 0,
     'noop_update_total': 0,
     'is_throttled': False,
     'throttle_time_in_millis': 0,
     'write_load': 2.3046649255828098e-06},
    'get': {'total': 0,
     'time_in_millis': 0,
     'exists_total': 0,
     'exists_time_in_millis': 0,
     'missing_total': 0,
     'missing_time_in_millis': 0,
     'current': 0},
    'search': {'open_contexts': 0,
     'query_total': 1795,
     'query_time_in_millis': 373,
     'query_current': 0,
     'fetch_total': 1787,
     'fetch_time_in_millis': 27,
     'fetch_current'

In [42]:
db.__dir__()

['embedding',
 'index_name',
 'query_field',
 'vector_query_field',
 'distance_strategy',
 'strategy',
 'client',
 '__module__',
 '__doc__',
 '__init__',
 'get_user_agent',
 'connect_to_elasticsearch',
 'embeddings',
 'similarity_search',
 'max_marginal_relevance_search',
 '_identity_fn',
 '_select_relevance_score_fn',
 'similarity_search_with_score',
 'similarity_search_by_vector_with_relevance_scores',
 '_search',
 'delete',
 '_create_index_if_not_exists',
 '_ElasticsearchStore__add',
 'add_texts',
 'add_embeddings',
 'from_texts',
 '_create_cls_from_kwargs',
 'from_documents',
 'ExactRetrievalStrategy',
 'ApproxRetrievalStrategy',
 'SparseVectorRetrievalStrategy',
 '__abstractmethods__',
 '_abc_impl',
 'adelete',
 'aadd_texts',
 'add_documents',
 'aadd_documents',
 'search',
 'asearch',
 '_euclidean_relevance_score_fn',
 '_cosine_relevance_score_fn',
 '_max_inner_product_relevance_score_fn',
 'asimilarity_search_with_score',
 '_similarity_search_with_relevance_scores',
 '_asimilarit

Here goes the load. I like how small the code is, but eventually I'd love to see more flexibility on how we model the data as I'd like to do more hybrid search techniques.

In [92]:
from pathlib import Path

count = 0
bookFilePath = "starwars_*_canon_data*.pickle"
files = sorted(Path('.').glob(bookFilePath))
batchtext = []
for fn in files:
    print(f"Starting book: {fn}")
    with open(fn,'rb') as f:
        part = pickle.load(f)

        for ix, (key, value) in tqdm(enumerate(part.items()), total=len(part)):
            paragraphs = value['paragraph']
            for px, p in enumerate(paragraphs):
                # print(f"{ix} {px} {title}")
                batchtext.append(p)
                count = count + 1

print("")
print(len(batchtext))
db.from_texts(batchtext, embedding=hf, index_name=index_name, es_url=ES_URL , es_api_key=API_KEY)

Starting book: starwars_small_canon_data.pickle


100%|██████████| 3/3 [00:00<00:00, 3778.65it/s]


651


Now we create a prompt chain that gets the most relevant passage from Elasticsearch using a vector search, and then uses that knowledge in the prompt to the LLM.

In [93]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM

topic = "Star Wars"
index_name = "book_wookieepedia_small"

cache_dir = "./cache"

def getFlanLarge():
    model_id = 'google/flan-t5-large'
    print(f">> Prep. Get {model_id} ready to go")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id, cache_dir=cache_dir)

    pipe = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=100
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

def make_the_llm():
    template_informed = """
    I am a helpful AI that answers questions. When I don't know the answer I say I don't know.
    I know context: {context}
    when asked: {question}
    my response using only information in the context is: """

    prompt_informed = PromptTemplate(template=template_informed, input_variables=["context", "question"])

    llm = getFlanLarge()

    return LLMChain(prompt=prompt_informed, llm=llm)

llm_chain_informed= make_the_llm()




>> Prep. Get google/flan-t5-large ready to go


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Time to win at Star Wars trivia

As long as the questions are about Ashoka or Din Djarin, you can now ask questions!  Here's some questions you could try.  Note that ChatGPT 3.5 and GPT4 hallucinate bad answers to these questions:

* Who helped the Mandalorian build his N1 Starfighter?
* Who trained Ahsoka when she was a youngling?

In [94]:

## how to ask a question
def ask_a_question(question):
    similar_docs = db.similarity_search(question)
    print(f'The most relevant passage: \n\t{similar_docs[0].page_content}')

    ## Ask Local LLM context informed prompt
    informed_context= similar_docs[0].page_content
    informed_response = llm_chain_informed.run(context=informed_context,question=question)

    return informed_response

# The conversational loop

print("""


.___  ___.      ___   ____    ____    .___________. __    __   _______     _  _    .___________. __    __
|   \/   |     /   \  \   \  /   /    |           ||  |  |  | |   ____|   | || |   |           ||  |  |  |
|  \  /  |    /  ^  \  \   \/   /     `---|  |----`|  |__|  | |  |__      | || |_  `---|  |----`|  |__|  |
|  |\/|  |   /  /_\  \  \_    _/          |  |     |   __   | |   __|     |__   _|     |  |     |   __   |
|  |  |  |  /  _____  \   |  |            |  |     |  |  |  | |  |____       | |       |  |     |  |  |  |
|__|  |__| /__/     \__\  |__|            |__|     |__|  |__| |_______|      |_|       |__|     |__|  |__|

.______    _______    ____    __    ____  __  .___________. __    __     ____    ____  ______    __    __
|   _  \  |   ____|   \   \  /  \  /   / |  | |           ||  |  |  |    \   \  /   / /  __  \  |  |  |  |
|  |_)  | |  |__       \   \/    \/   /  |  | `---|  |----`|  |__|  |     \   \/   / |  |  |  | |  |  |  |
|   _  <  |   __|       \            /   |  |     |  |     |   __   |      \_    _/  |  |  |  | |  |  |  |
|  |_)  | |  |____       \    /\    /    |  |     |  |     |  |  |  |        |  |    |  `--'  | |  `--'  |
|______/  |_______|       \__/  \__/     |__|     |__|     |__|  |__|        |__|     \______/   \______/



""")

print(f'I am a trivia chat bot, ask me any question about {topic}')

while True:
    question = input("User Question >> ")
    response= ask_a_question(question)
    print(f"\tAnswer  : {response}")





.___  ___.      ___   ____    ____    .___________. __    __   _______     _  _    .___________. __    __  
|   \/   |     /   \  \   \  /   /    |           ||  |  |  | |   ____|   | || |   |           ||  |  |  | 
|  \  /  |    /  ^  \  \   \/   /     `---|  |----`|  |__|  | |  |__      | || |_  `---|  |----`|  |__|  | 
|  |\/|  |   /  /_\  \  \_    _/          |  |     |   __   | |   __|     |__   _|     |  |     |   __   | 
|  |  |  |  /  _____  \   |  |            |  |     |  |  |  | |  |____       | |       |  |     |  |  |  | 
|__|  |__| /__/     \__\  |__|            |__|     |__|  |__| |_______|      |_|       |__|     |__|  |__| 
                                                                                                           
.______    _______    ____    __    ____  __  .___________. __    __     ____    ____  ______    __    __  
|   _  \  |   ____|   \   \  /  \  /   / |  | |           ||  |  |  |    \   \  /   / /  __  \  |  |  |  | 
|  |_)  | |  |__       \ 

KeyboardInterrupt: Interrupted by user

Have you fallen in love with LangChain as much as I have? As a wise old Jedi once said: “That's good. You have taken your first step into a larger world.” There are lots of directions to go from here. LangChain takes the complexity away from working with AI prompt engineering. I know Elasticsearch has many other roles to play here as long term memory for generative AI, so I am very excited to see what comes out of this quickly changing space.

## License

The example: `blog-langchain-elasticsearch` is available under the Apache 2.0 license.
For more details see [LICENSE](../LICENSE).